In [ ]:
import pandas as pd
import numpy as np
import random
from numpy.random import choice

# Sample data for courses
courses_data = {
    'Course': ['PDC', 'CNET', 'Numerical', 'AI', 'SE'],
    'Type': ['Theory', 'Theory', 'Theory', 'Lab', 'Theory'],
    'S.#': ['101', '102', '103', '104', '105'],
    'Section': ['A', 'B', 'F', 'C', 'S'],
    'Section-Strength': [30, 25, 30, 20, 35],
    'Professor': ['Ali', 'Zeeshan', 'Danish', 'Almas', 'Shafaq']
}
df = pd.DataFrame(courses_data)

# Sample data for rooms
rooms = ['C301', 'C302', 'A303', 'C404', 'Rawal 5']

# Sample data for timeslots
timeslots = ['08:30-10:10', '10:20-12:00', '12:10-13:50', '14:00-15:40', '15:50-17:30']

# Create chromosomes based on course + type + section + section-strength + professor + lecture details
chromosomes = np.array([
    [
        row['Course'],
        1 if row['Type'] == 'Lab' else 0,  # Binary encoding for Lab (1) and Theory (0)
        row['S.#']+'-'+row['Section'],
        row['Section-Strength'],
        row['Professor'],
        random.choice(range(1, 6)),  # First-lecture-day (randomly assigned)
        random.choice(timeslots),     # First-lecture-timeslot (randomly assigned)
        random.choice(rooms),         # First-lecture-room (randomly assigned)
        random.randint(60, 120),      # First-lecture-room-size (randomly assigned)
        random.choice(range(1, 6)),  # Second-lecture-day (randomly assigned)
        random.choice(timeslots),     # Second-lecture-timeslot (randomly assigned)
        random.choice(rooms),         # Second-lecture-room (randomly assigned)
        random.randint(60, 120)       # Second-lecture-room-size (randomly assigned)
    ]
    for _, row in df.iterrows()
])

populationSize = 30
generations = 100000
mutationProb = 0.2

# Generate initial population
population = [np.array([[random.choice(chromosomes) for _ in timeslots] for _ in rooms])  for _ in range(populationSize)]

def fitness(timetable):
    conflicts = 0

    # Hard Constraints...

    # Constraint: Classes can only be scheduled in free classrooms.
    # Penalize if a room is assigned to more than one section at the same time.
    for room_schedule in timetable:
        for day_schedule in room_schedule.T:
            for lecture in day_schedule:
                if len(lecture) < 8:  # Ensure lecture has enough elements
                    continue
                sections_in_slot = [other_lecture[2] for other_lecture in day_schedule if len(other_lecture) >= 8]
                unique_sections = set(sections_in_slot)
                conflicts += len(sections_in_slot) - len(unique_sections)

    # Constraint: A classroom should be big enough to accommodate the section.
    # Penalize if a section requires a room larger than available.
    for room_schedule in timetable:
        for day_schedule in room_schedule.T:
            for lecture in day_schedule:
                if len(lecture) < 8:  # Ensure lecture has enough elements
                    continue
                room_size = lecture[-1]
                section_strength = lecture[3]
                if room_size < section_strength:
                    conflicts += 1

    # Constraint: A professor should not be assigned two different lectures at the same time.
    # Penalize if a professor teaches more than one course at the same time.
    for day_schedule in timetable.T:
        for lecture in day_schedule:
            if len(lecture) < 8:  # Ensure lecture has enough elements
                continue
            professor = lecture[4]
            courses_assigned = [other_lecture[0] for other_lecture in day_schedule if len(other_lecture) >= 8]
            if courses_assigned.count(lecture[0]) > 1:
                conflicts += 1

    # Constraint: The same section cannot be assigned to two different rooms at the same time.
    # Penalize if a section is scheduled in more than one room at the same time.
    for day_schedule in timetable.T:
        for lecture in day_schedule:
            if len(lecture) < 8:  # Ensure lecture has enough elements
                continue
            section = lecture[2]
            rooms_assigned = [other_lecture[7] for other_lecture in day_schedule if len(other_lecture) >= 8]
            if rooms_assigned.count(lecture[7]) > 1:
                conflicts += 1

    # Constraint: A room cannot be assigned for two different sections at the same time.
    # Penalize if a room is scheduled for more than one section at the same time.
    for room_schedule in timetable:
        for day_schedule in room_schedule.T:
            for lecture in day_schedule:
                if len(lecture) < 8:  # Ensure lecture has enough elements
                    continue
                room = lecture[7]
                sections_assigned = [other_lecture[2] for other_lecture in day_schedule if len(other_lecture) >= 8]
                if sections_assigned.count(lecture[2]) > 1:
                    conflicts += 1

    # Constraint: No professor can teach more than 3 courses.
    # Penalize if a professor teaches more than 3 courses.
    for day_schedule in timetable.T:
        for lecture in day_schedule:
            if len(lecture) < 8:  # Ensure lecture has enough elements
                continue
            professor = lecture[4]
            courses_taught = [other_lecture[0] for other_lecture in day_schedule if len(other_lecture) >= 8]
            if courses_taught.count(lecture[0]) > 3:
                conflicts += 1

    # Constraint: No section can have more than 5 courses in a semester.
    # Penalize if a section has more than 5 courses scheduled.
    for day_schedule in timetable.T:
        for lecture in day_schedule:
            if len(lecture) < 8:  # Ensure lecture has enough elements
                continue
            section = lecture[2]
            courses_assigned = [other_lecture[0] for other_lecture in day_schedule if len(other_lecture) >= 8]
            if courses_assigned.count(lecture[0]) > 5:
                conflicts += 1

    # Constraint: Each course would have two lectures per week not on the same or adjacent days.
    # Penalize if a course has lectures scheduled on the same or adjacent days.
    for day_schedule in timetable.T:
        for lecture in day_schedule:
            if len(lecture) < 8:  # Ensure lecture has enough elements
                continue
            course = lecture[0]
            days = [other_lecture[5] for other_lecture in day_schedule if len(other_lecture) >= 8 and other_lecture[0] == course]
            unique_days = set(days)
            if len(unique_days) != 2:
                conflicts += 1
            else:
                sorted_days = sorted(unique_days)
                if sorted_days[1] - sorted_days[0] <= 1:
                    conflicts += 1

    # Constraint: Lab lectures should be conducted in two consecutive slots.
    # Penalize if a lab lecture is not scheduled in two consecutive slots.
    for room_schedule in timetable:
        for day_schedule in room_schedule.T:
            for i, lecture in enumerate(day_schedule[:-1]):
                if len(lecture) < 8 or len(day_schedule[i + 1]) < 8:  # Ensure lectures have enough elements
                    continue
                if lecture[1] == 1 and day_schedule[i + 1][1] != 1:  # Check if it's a lab lecture and next lecture is not
                    conflicts += 1

    # Soft Constraints...

    # Constraint: All the theory classes should be taught in the morning session and all the lab sessions should be done in the afternoon session.
    # Penalize if a theory class is scheduled in the afternoon session or a lab session is scheduled in the morning session.
    for room_schedule in timetable:
        for day_schedule in room_schedule.T:
            for lecture in day_schedule:
                if len(lecture) < 8:  # Ensure lecture has enough elements
                    continue
                if lecture[1] == 0 and int(lecture[6].split(':')[0]) >= 14:  # Theory class in afternoon session
                    conflicts += 1
                elif lecture[1] == 1 and int(lecture[6].split(':')[0]) < 14:  # Lab class in morning session
                    conflicts += 1

    # Constraint: A class should be held in the same classroom across the whole week.
    # Penalize if a class is scheduled in different classrooms across the week.
    for room_schedule in timetable:
        for day_schedule in room_schedule.T:
            for lecture in day_schedule:
                if len(lecture) < 8:  # Ensure lecture has enough elements
                    continue
                rooms_assigned = [other_lecture[7] for other_lecture in day_schedule if len(other_lecture) >= 8]
                unique_rooms = set(rooms_assigned)
                if len(unique_rooms) != 1:
                    conflicts += 1

    # Constraint: Teachers may prefer longer blocks of continuous teaching time to minimize interruptions and maximize productivity except when the courses are different.
    # Penalize if a teacher's lectures are not scheduled in longer continuous blocks.
    for day_schedule in timetable.T:
        for lecture in day_schedule:
            if len(lecture) < 8:  # Ensure lecture has enough elements
                continue
            professor = lecture[4]
            lectures = [other_lecture for other_lecture in day_schedule if len(other_lecture) >= 8 and other_lecture[4] == professor]
            sorted_lectures = sorted(lectures, key=lambda x: (x[5], x[6]))  # Sort by day and timeslot
            consecutive_blocks = 1
            for i in range(1, len(sorted_lectures)):
                if sorted_lectures[i][5] == sorted_lectures[i - 1][5]:  # Same day
                    if timeslots.index(sorted_lectures[i][6]) - timeslots.index(sorted_lectures[i - 1][6]) == 1:  # Adjacent timeslots
                        consecutive_blocks += 1
                    else:
                        conflicts += 1  # Penalize if not adjacent timeslots
                else:
                    if consecutive_blocks < 4:  # Penalize if consecutive blocks are less than 4
                        conflicts += 1
                    consecutive_blocks = 1


    return -conflicts  # Return negative sum of conflicts



def initialize_population(population_size):
    population = []
    for _ in range(population_size):
        shuffled_chromosomes = np.random.permutation(chromosomes)
        timetable = np.array([[shuffled_chromosomes[i] for _ in timeslots] for i in range(len(rooms))])
        population.append(timetable)
    return population


def selection(index, scores):
    # Roulette wheel selection based on index and respective scores
    distribution = scores / scores.sum()
    return choice(index, 2, p=distribution, replace=False)

def crossover(parent1, parent2):
    # Single point crossover
    if random.random() < 0.5:
        index = random.randint(1, len(parent1)-2)
        child1 = np.concatenate((parent1[:index], parent2[index:]))
        child2 = np.concatenate((parent2[:index], parent1[index:]))
    else:
        index = random.randint(1, len(timeslots)-2)
        child1 = np.concatenate((parent1[:, :index], parent2[:, index:]), axis=1)
        child2 = np.concatenate((parent1[:, :index], parent2[:, index:]), axis=1)
    return [child1, child2]

def tournament_selection(scores, k=3):
    selected_indices = []
    for _ in range(2):
        candidates = np.random.choice(len(scores), size=k, replace=False)
        winner_index = candidates[np.argmax(scores[candidates])]
        selected_indices.append(winner_index)
    return selected_indices

def mutation(child, probability):
    for i in range(len(child)):
        if random.random() < probability:
            child[i] = random.choice(chromosomes)
    return child

def evolution(population, generations):
    best_timetable = None
    best_score = float('-inf')

    for gen in range(generations):
        scores = np.array([fitness(p) for p in population])
        new_population = []

        for _ in range(populationSize // 2):
            parents = tournament_selection(scores)
            children = crossover(population[parents[0]], population[parents[1]])
            children[0] = mutation(children[0], mutationProb)
            children[1] = mutation(children[1], mutationProb)
            new_population.extend(children)

        population = sorted(population + new_population, key=fitness, reverse=True)[:populationSize]

        if gen % 100 == 0:
            print(f"Generation - {gen + 1}")
            print(f"Best Score: {fitness(population[0])}")
            if best_score < fitness(population[0]):
                best_score = fitness(population[0])
                best_timetable = population[0]

            # Print the current best timetable
            print_timetable(best_timetable)

            # Check if the best timetable fulfills all conditions
            if checkFinal(population[0]):
                print("Timetable found meeting all constraints!")
                return population[0]  # Return the best timetable

    print("Reached maximum generations.")
    return best_timetable



def checkFinal(timetable):
    # Soft Constraints...
    for room_schedule in timetable:
        for day_schedule in room_schedule.T:
            for lecture in day_schedule:
                if len(lecture) < 8:  # Ensure lecture has enough elements
                    continue
                if lecture[1] == 0 and int(lecture[6].split(':')[0]) >= 14:  # Theory class in afternoon session
                    return False
                elif lecture[1] == 1 and int(lecture[6].split(':')[0]) < 14:  # Lab class in morning session
                    return False

    for room_schedule in timetable:
        for day_schedule in room_schedule.T:
            rooms_assigned = [other_lecture[7] for other_lecture in day_schedule if len(other_lecture) >= 8]
            unique_rooms = set(rooms_assigned)
            if len(unique_rooms) != 1:
                return False

    for day_schedule in timetable.T:
        for lecture in day_schedule:
            if len(lecture) < 8:  # Ensure lecture has enough elements
                continue
            professor = lecture[4]
            lectures = [other_lecture for other_lecture in day_schedule if len(other_lecture) >= 8 and other_lecture[4] == professor]
            sorted_lectures = sorted(lectures, key=lambda x: (x[5], x[6]))  # Sort by day and timeslot
            consecutive_blocks = 1
            for i in range(1, len(sorted_lectures)):
                if sorted_lectures[i][5] == sorted_lectures[i - 1][5]:  # Same day
                    if timeslots.index(sorted_lectures[i][6]) - timeslots.index(sorted_lectures[i - 1][6]) == 1:  # Adjacent timeslots
                        consecutive_blocks += 1
                    else:
                        return False  # Penalize if not adjacent timeslots
                else:
                    if consecutive_blocks < 4:  # Penalize if consecutive blocks are less than 4
                        return False
                    consecutive_blocks = 1

    return True  # Return True if all soft constraints are met

def print_timetable(timetable):
    if timetable is not None:
        final_time_table = pd.DataFrame([[" | ".join(cell) for cell in row] for row in timetable], columns=timeslots, index=rooms)
        print(final_time_table)
    else:
        print("No valid timetable found.")

# Run the genetic algorithm
final_population = evolution(population, generations)




Streaming output truncated to the last 5000 lines.
C302     PDC | 0 | 101-A | 30 | Ali | 4 | 15:50-17:30 |...   
A303     SE | 0 | 105-S | 35 | Shafaq | 4 | 10:20-12:00...   
C404     PDC | 0 | 101-A | 30 | Ali | 4 | 15:50-17:30 |...   
Rawal 5  SE | 0 | 105-S | 35 | Shafaq | 4 | 10:20-12:00...   

                                               10:20-12:00  \
C301     PDC | 0 | 101-A | 30 | Ali | 4 | 15:50-17:30 |...   
C302     PDC | 0 | 101-A | 30 | Ali | 4 | 15:50-17:30 |...   
A303     SE | 0 | 105-S | 35 | Shafaq | 4 | 10:20-12:00...   
C404     PDC | 0 | 101-A | 30 | Ali | 4 | 15:50-17:30 |...   
Rawal 5  SE | 0 | 105-S | 35 | Shafaq | 4 | 10:20-12:00...   

                                               12:10-13:50  \
C301     PDC | 0 | 101-A | 30 | Ali | 4 | 15:50-17:30 |...   
C302     PDC | 0 | 101-A | 30 | Ali | 4 | 15:50-17:30 |...   
A303     SE | 0 | 105-S | 35 | Shafaq | 4 | 10:20-12:00...   
C404     PDC | 0 | 101-A | 30 | Ali | 4 | 15:50-17:30 |...   
Rawal 5  SE | 0 |